In [1]:
import numpy as np
from scipy import stats

In [2]:
y = 1.0 * np.random.choice(2, size = 200)
mvn = stats.multivariate_normal.rvs
X = np.array([mvn([1, 1] if heads else [4, 4]) for heads in y])

In [16]:
#import lr
import newton
import irls
import logistic

reload(logistic)
clf = logistic
clf.minimize = newton.minimize
model = clf.fit(X, y)
y_hat = clf.predict(model, X)
print np.mean(y == y_hat)

0.585


In [17]:
clf = logistic.Classifier()
model = clf.fit(X, y)
print '%0.3f' % np.mean(np.mean(clf.predict(X) != y))

0.415
